# Hyperparameter tunning.
We are starting this notebook in order to have a full view of tensorboard. And to use it as a parameter tunning tool.

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

Clear any logs from previous runs

    rm -rf ./logs/

In [2]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

## Loading the Data.

In [21]:
import librosa
import librosa.display

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
from datetime import datetime
import re

In [14]:
files_list = os.listdir('../Datasets/s_a_d__datasets/audio_esc50')
paths_list = []
for file in files_list:
    paths_list.append('../Datasets/s_a_d__datasets/audio_esc50/'+file)
    
files_list_csv = os.listdir('../Datasets/s_a_d__datasets/experimental')

In [15]:
Images = []
Labels = []

for file in files_list_csv:
    image_to_be = pd.read_csv('../Datasets/s_a_d__datasets/experimental/'+file)
    Images.append(image_to_be.to_numpy())
    Labels.append(re.split(r'-|\.', file)[3])
    
Images = np.asarray(Images)
Labels = np.asarray(Labels)

Labels = pd.get_dummies(Labels)

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(Images, Labels, test_size=0.3, stratify=Labels)

X_train = X_train.reshape(1400, 128, 217, 1)
X_test = X_test.reshape(600, 128, 217, 1)


input_dim = (128,217,1)

Following we have the creation of the hyperparameter lists. Using some hiperparameters.

In [29]:
HP_NUM_FILTERS = hp.HParam('num_filters', hp.Discrete([16, 32, 64]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.5))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'RMSprop']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
    hparams=[HP_NUM_FILTERS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )


In [19]:
for optimizer in HP_OPTIMIZER.domain.values:
    print(optimizer)

RMSprop
adam


In [22]:
# Create a TensorBoard callback
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')


In [23]:
def train_test_model(hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_NUM_FILTERS],(3,3), padding='same', activation=tf.nn.relu, input_shape=input_dim),
        tf.keras.layers.Conv2D(hparams[HP_NUM_FILTERS],(3,3), padding='same', activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(2*hparams[HP_NUM_FILTERS],(3,3), padding='same', activation=tf.nn.relu),
        tf.keras.layers.Conv2D(2*hparams[HP_NUM_FILTERS],(3,3), padding='same', activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(4*hparams[HP_NUM_FILTERS],(3,3), padding='same', activation=tf.nn.relu),
        tf.keras.layers.Conv2D(4*hparams[HP_NUM_FILTERS],(3,3), padding='same', activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(1024, activation=tf.nn.relu),
        tf.keras.layers.Dense(50, activation=tf.nn.softmax),
    ])
    model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='categorical_crossentropy',
      metrics=['accuracy'],
    )

    model.fit(X_train, y_train, epochs=70, batch_size=30, callbacks=[tboard_callback])
    _, accuracy = model.evaluate(X_test, y_test)
    return accuracy

In [24]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [30]:
session_num = 0

for num_filnum_filtersters in HP_NUM_FILTERS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_FILTERS: num_filters,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,
                }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1


--- Starting trial: run-0
{'num_filters': 16, 'dropout': 0.1, 'optimizer': 'RMSprop'}
Epoch 1/70
47/47 [==============================] - 36s 763ms/step - loss: 3.8647 - accuracy: 0.0293
Epoch 2/70
47/47 [==============================] - 36s 763ms/step - loss: 3.7196 - accuracy: 0.0557
Epoch 3/70
47/47 [==============================] - 36s 764ms/step - loss: 3.4771 - accuracy: 0.0821
Epoch 4/70
47/47 [==============================] - 36s 765ms/step - loss: 3.2664 - accuracy: 0.1250
Epoch 5/70
47/47 [==============================] - 40s 855ms/step - loss: 3.1374 - accuracy: 0.1357
Epoch 6/70
47/47 [==============================] - 40s 844ms/step - loss: 3.0137 - accuracy: 0.1607
Epoch 7/70
47/47 [==============================] - 40s 845ms/step - loss: 2.8998 - accuracy: 0.1800
Epoch 8/70
47/47 [==============================] - 40s 844ms/step - loss: 2.7938 - accuracy: 0.2186
Epoch 9/70
47/47 [==============================] - 40s 844ms/step - loss: 2.6714 - accuracy: 0.2379
Epoch

Epoch 9/70
47/47 [==============================] - 39s 839ms/step - loss: 2.7835 - accuracy: 0.2150
Epoch 10/70
47/47 [==============================] - 39s 838ms/step - loss: 2.7031 - accuracy: 0.2171
Epoch 11/70
47/47 [==============================] - 40s 842ms/step - loss: 2.5676 - accuracy: 0.2486
Epoch 12/70
47/47 [==============================] - 40s 843ms/step - loss: 2.4200 - accuracy: 0.2771
Epoch 13/70
47/47 [==============================] - 39s 839ms/step - loss: 2.3691 - accuracy: 0.2857
Epoch 14/70
47/47 [==============================] - 40s 841ms/step - loss: 2.2276 - accuracy: 0.3257
Epoch 15/70
47/47 [==============================] - 39s 840ms/step - loss: 2.1573 - accuracy: 0.3543
Epoch 16/70
47/47 [==============================] - 40s 841ms/step - loss: 2.0290 - accuracy: 0.3879
Epoch 17/70
47/47 [==============================] - 40s 841ms/step - loss: 1.9768 - accuracy: 0.3950
Epoch 18/70
47/47 [==============================] - 39s 840ms/step - loss: 1.8603 

47/47 [==============================] - 39s 834ms/step - loss: 2.2924 - accuracy: 0.2964
Epoch 26/70
47/47 [==============================] - 39s 836ms/step - loss: 2.2277 - accuracy: 0.3264
Epoch 27/70
47/47 [==============================] - 39s 835ms/step - loss: 2.1404 - accuracy: 0.3293
Epoch 28/70
47/47 [==============================] - 39s 837ms/step - loss: 2.0771 - accuracy: 0.3593
Epoch 29/70
47/47 [==============================] - 39s 835ms/step - loss: 2.0677 - accuracy: 0.3557
Epoch 30/70
47/47 [==============================] - 39s 834ms/step - loss: 2.0171 - accuracy: 0.3500
Epoch 31/70
47/47 [==============================] - 39s 836ms/step - loss: 2.0174 - accuracy: 0.3693
Epoch 32/70
47/47 [==============================] - 39s 835ms/step - loss: 1.9117 - accuracy: 0.3843
Epoch 33/70
47/47 [==============================] - 39s 833ms/step - loss: 1.9202 - accuracy: 0.4043
Epoch 34/70
47/47 [==============================] - 39s 837ms/step - loss: 1.8643 - accuracy:

47/47 [==============================] - 89s 2s/step - loss: 0.5678 - accuracy: 0.8221
Epoch 46/70
47/47 [==============================] - 89s 2s/step - loss: 0.4179 - accuracy: 0.8600
Epoch 47/70
47/47 [==============================] - 89s 2s/step - loss: 0.4199 - accuracy: 0.8593
Epoch 48/70
47/47 [==============================] - 89s 2s/step - loss: 0.3771 - accuracy: 0.8707
Epoch 49/70
47/47 [==============================] - 89s 2s/step - loss: 0.3736 - accuracy: 0.8786
Epoch 50/70
47/47 [==============================] - 89s 2s/step - loss: 0.3908 - accuracy: 0.8729
Epoch 51/70
47/47 [==============================] - 89s 2s/step - loss: 0.3260 - accuracy: 0.8864
Epoch 52/70
47/47 [==============================] - 89s 2s/step - loss: 0.2766 - accuracy: 0.9114
Epoch 53/70
47/47 [==============================] - 89s 2s/step - loss: 0.2740 - accuracy: 0.9029
Epoch 54/70
47/47 [==============================] - 89s 2s/step - loss: 0.2404 - accuracy: 0.9186
Epoch 55/70
47/47 [===

47/47 [==============================] - 89s 2s/step - loss: 0.6592 - accuracy: 0.7729
Epoch 68/70
47/47 [==============================] - 89s 2s/step - loss: 0.6253 - accuracy: 0.7929
Epoch 69/70
47/47 [==============================] - 89s 2s/step - loss: 0.6549 - accuracy: 0.7914
Epoch 70/70
19/19 [==============================] - 8s 407ms/step - loss: 1.7995 - accuracy: 0.5833
--- Starting trial: run-8
{'num_filters': 64, 'dropout': 0.1, 'optimizer': 'RMSprop'}
Epoch 1/70
47/47 [==============================] - 235s 5s/step - loss: 4.1580 - accuracy: 0.0307
Epoch 2/70
47/47 [==============================] - 237s 5s/step - loss: 3.7625 - accuracy: 0.0493
Epoch 3/70
47/47 [==============================] - 235s 5s/step - loss: 3.5950 - accuracy: 0.0543
Epoch 4/70
47/47 [==============================] - 235s 5s/step - loss: 3.3725 - accuracy: 0.0879
Epoch 5/70
47/47 [==============================] - 235s 5s/step - loss: 3.1820 - accuracy: 0.1336
Epoch 6/70
47/47 [===============

47/47 [==============================] - 235s 5s/step - loss: 2.2663 - accuracy: 0.3143
Epoch 16/70
47/47 [==============================] - 235s 5s/step - loss: 2.1300 - accuracy: 0.3543
Epoch 17/70
47/47 [==============================] - 235s 5s/step - loss: 2.1160 - accuracy: 0.3586
Epoch 18/70
47/47 [==============================] - 235s 5s/step - loss: 1.9914 - accuracy: 0.3821
Epoch 19/70
47/47 [==============================] - 235s 5s/step - loss: 1.9241 - accuracy: 0.3921
Epoch 20/70
47/47 [==============================] - 235s 5s/step - loss: 1.8636 - accuracy: 0.4193
Epoch 21/70
47/47 [==============================] - 235s 5s/step - loss: 1.8206 - accuracy: 0.4514
Epoch 22/70
47/47 [==============================] - 235s 5s/step - loss: 1.7249 - accuracy: 0.4679
Epoch 23/70
47/47 [==============================] - 235s 5s/step - loss: 1.7038 - accuracy: 0.4700
Epoch 24/70
47/47 [==============================] - 235s 5s/step - loss: 1.6386 - accuracy: 0.4857
Epoch 25/70


In [3]:
%tensorboard --logdir logs/hparam_tuning

In [31]:
for num_filters in HP_NUM_FILTERS.domain.values:
    print(num_filters)

16
32
64
